In [1]:
import time

import numpy as np
# import torch

import tools.rotations as rot
# import pytorch_kinematics as pk

from controller import locomotion
from control_interface_real_robot import real_robot
import mujoco
import cvxpy as cp
from control_interface_real_robot import q_sim2real

import copy
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.executable

'/usr/bin/python3'

In [3]:
hand_real = real_robot(control_mode='position', legs=5)

leg num: 5


In [4]:
hand_real.update_data()
hand_real.q

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
q0 = np.array([-1.62289571e-04, -3.50667052e-04,  7.98538815e-02,  9.99999738e-01,
       -4.46340938e-04,  5.67392530e-04, -4.89152197e-05, 
               -0.7,   7.91144283e-01,  0.3,  0.3,
               0,  7.93214611e-01,  0.3,  0.3,  0.00000000e+00,
               -1.47079633e+00, -8.00000000e-01, -8.00000000e-01, 
               -6.26414686e-06,  7.93034999e-01,  0.3,  0.3,
               -1.00000000e+00,  7.92384667e-01,  0.3,  0.3])
q0.shape

(27,)

In [6]:
# test pytorch kinematics
xml_path = 'descriptions/five_finger_hand_ssss.xml'
# chain = pk.build_chain_from_mjcf('descriptions/single_finger_ssss.xml', 'finger_1')
# print(chain)
var_num = 12
q0 = np.array([-1.62289571e-04, -3.50667052e-04,  7.98538815e-02,  9.99999738e-01,
       -4.46340938e-04,  5.67392530e-04, -4.89152197e-05, 
               0,   7.91144283e-01,  0.3,  0.3,
               -0.4,  7.93034999e-01,  0.3,  0.3, 
               0,   -1.4,  -1,  -1,  
              0,   -1.4,  -1,  -1, 
              0.3,  7.92384667e-01,  0.3,  0.3])
r = locomotion(xml_path, var_num=var_num, q0=q0)

print('locomotion freq', r.f)
# take a rest to let the acceleration to zero
t0 = time.time()
q0 = np.copy(r.q)
while 1:
    # r.step()
    # r.send_torque(r.C)
    r.joint_computed_torque_control(q0)
    if time.time() - t0 > 2:
        break
xh_local_init = copy.deepcopy(r.xh_local) 

Kinematic model has been loaded from kinematics/q2pose.txt
locomotion freq 1.0


In [ ]:
xh_local_init

In [ ]:
hand_real.q

In [ ]:
# q0 = np.array([ 0.13345633,   0.69029135,  0.41264084 , 
#                 -0.3,  0.69029135,  0.41264084,
#                 -0.00153398,   0.69029135,  0.41264084, 
#                 -0.11965051,  0.69029135,  0.41264084,
#                 1,   0.69029135,  0.41264084])
# 
# 
# # q_cmd = q_sim2real(q0, legs=5)
# hand_real.move_to_joints(q0.reshape(1,-1), sim2real=False)  # move to init pose

In [ ]:
hand_real.update_data()
hand_real.q

In [ ]:
# q_cmd = np.zeros(12)
# q_cmd[0] = 0.5
# hand_real.move_to_joints(q_cmd)

# q_cmd_sim = np.zeros(20)
# q_cmd_sim[13] = 0 
# q_cmd_sim[17] = 0
# r.modify_joint(q_cmd_sim)

# q_cmd = q_sim2real(q0, legs=5)
# 
# hand_real.move_to_joints(q_cmd, sim2real=False)  # move to init pose
# hand_real.send_position(q_cmd)
# hand_real.move_to_joints_by_vel(q_cmd)


In [28]:
xy_step

array([ 0.   , -0.005])

In [29]:
## gait design for four legs
 # this is the init position


t0 = time.time()
r.f = 1
T = 1 / r.f
v = np.array([0.0, -0.02,0]) 
# v = np.array([0.04,0 ,0]) 
# v = np.array([0, -0.02,0])
h = 0.05  # z axis lifting height for swing legs
r.vel_cmd[:3] = v

xy_step = v[:2] * T/2 * 0.5 # based the MIT paper
x_cmd = copy.deepcopy(xh_local_init)
x_cmd = [a[:3] for a in x_cmd]  # position command for legs
q_cmd_list = []
t_total = 10 * T

k = 0
q_desired = copy.deepcopy(r.q)

while 1:
    
    tau_Grf = np.zeros(20)
    t_now  = time.time() - t0
    t_period = t_now % (1/r.f)  # [0, T]
    
    if t_period < T/3: # FL and BR, front left and back right legs move during [0, T/2]
        swing_legs = [0,]
        stance_legs = [1, 4]
    elif T/3 <=t_period < T/3*2 :
        swing_legs = [1]
        stance_legs = [4, 0]
    else:
        swing_legs = [4]
        stance_legs = [0, 1]
    # update vel direction in hand base frame
    # xy_step = rot.quat2mat(r.x[3:]).T @ np.array([xy_step[0], xy_step[1], 0])
    # xy_step = xy_step[:2]
    for i in swing_legs:
        t_tmp = t_period % (T/3) 
        x_cmd[i][:2] = r.bezier(t_tmp/ (T/3), xh_local_init[i][:2] - xy_step, xh_local_init[i][:2] + xy_step)
        if t_tmp <= T/6:
            x_cmd[i][2] = r.bezier(t_tmp /(T/6), xh_local_init[i][2], xh_local_init[i][2] + h)  # z axis
        else:
            x_cmd[i][2] = r.bezier((t_tmp - T/6) /(T/6), xh_local_init[i][2] + h, xh_local_init[i][2])
    
    for flag, i in enumerate(stance_legs):
        t_tmp = t_period % (T/3)
        x_cmd[i][:2] = r.bezier(t_tmp / (T/3), xh_local_init[i][:2] + xy_step * flag, xh_local_init[i][:2] - xy_step * (1-flag))
        x_cmd[i][2] = xh_local_init[i][2]
    
    q_desired = r.hand_kine.ik(x_cmd, q_init=q_desired)
    q_desired[8:16] = q0[8:16]
    r.joint_computed_torque_control(q_desired)
    # k += 1
    # q_desired[8:12] = q0[8:12]
    q_cmd = q_sim2real(q_desired, legs=5)
    q_cmd_list.append(q_cmd)
    # hand_real.send_position(q_cmd)

    if t_now > t_total:
        break

q_cmd_list = np.vstack(q_cmd_list)
q_cmd_list.shape

(14596, 15)

In [30]:
q_cmd_list = np.vstack(q_cmd_list)
q_cmd_list.shape
hand_real.move_to_joints(q_cmd_list[0,:].reshape(1,-1), sim2real=False)  # move to init pose

In [33]:
hand_real.move_to_joints(q_cmd_list, t=t_total)  # send to the real robot






In [ ]:
def generate_traj(v=np.array([0.0, -0.04,0]), w=None, f = 2, num=1):
    """
    generate trajctory for moving forward or rotation    
    :param v: 
    :param w: 
    :param f: frequency
    :param num: number of period
    :return: 
    """    
    T = 1 / f
    # v = np.array([0.0, -0.04,0]) 
    # # v = np.array([0.04,0 ,0]) 
    # # v = np.array([0, -0.02,0])
    h = 0.04  # z axis lifting height for swing legs
    
    if w is None:
        xy_step = [v[:2] * T/2 * 0.5] * 5  # based the MIT paper
    else:
        theta = w * T/2 * 0.5
        R = np.array([[np.cos(theta), -np.sin(theta)], 
                      [np.sin(theta), np.cos(theta)]])
        xy_step = [xh_local_init[i][:2] @ R - xh_local_init[i][:2] for i in range(5)]
        
    x_cmd = copy.deepcopy(xh_local_init)
    x_cmd = [a[:3] for a in x_cmd]  # position command for legs
    q_desired = copy.deepcopy(q0)
    
    dt = 0.002
    q_len = int(num * T / dt)
    q_cmd_list = []
    for a in range(q_len):
        t_now  = a * dt
        t_period = t_now % (1/r.f)  # [0, T]
        
        if t_period < T/2: # FL and BR, front left and back right legs move during [0, T/2]
            swing_legs = [0, 2, 3]
            stance_legs = [1, 4]
            # swing_legs = [0, 1]
            # stance_legs = [3, 4]
        else:
            swing_legs = [1, 4]
            stance_legs = [0, 2, 3]
            # swing_legs = [3, 4]
            # stance_legs = [0, 1]
        
        # update vel direction in hand base frame
        # xy_step = rot.quat2mat(r.x[3:]).T @ np.array([xy_step[0], xy_step[1], 0])
        # xy_step = xy_step[:2]
        for i in swing_legs:
            t_tmp = t_period % (T/2)
            x_cmd[i][:2] = r.bezier(t_tmp, xh_local_init[i][:2] - xy_step[i], xh_local_init[i][:2] + xy_step[i])
            if t_tmp <= T/4:
                x_cmd[i][2] = r.bezier(t_tmp /(T/4), xh_local_init[i][2], xh_local_init[i][2] + h)  # z axis
            else:
                x_cmd[i][2] = r.bezier((t_tmp - T/4) /(T/4), xh_local_init[i][2] + h, xh_local_init[i][2])
        
        for i in stance_legs:
            t_tmp = t_period % (T/2)
            x_cmd[i][:2] = r.bezier(t_tmp /(T/2), xh_local_init[i][:2] + xy_step[i], xh_local_init[i][:2] - xy_step[i])
            x_cmd[i][2] = xh_local_init[i][2]
        
        q_desired = r.hand_kine.ik(x_cmd, q_init=q_desired)
        q_cmd = q_sim2real(q_desired, legs=5)
        q_cmd_list.append(q_cmd)
    q_cmd_list = np.vstack(q_cmd_list)
    print('q_cmd_list', q_cmd_list.shape)
    
    return q_cmd_list


In [ ]:
q_cmd_list = generate_traj()

In [ ]:
q_cmd_list = generate_traj()
hand_real.move_to_joints(q_cmd_list[0,:].reshape(1,-1), sim2real=False)  # move to init pose

In [ ]:
# moving along straight line
f = 2
num = 5
t_total = num/f
q_cmd_list = generate_traj(v=np.array([0.0, 0.04,0]), f=2, num=num)  # generate traj

hand_real.move_to_joints(q_cmd_list, t=t_total)  # send to the real robot
time.sleep(t_total)

q_cmd = q_sim2real(q0, legs=5)
hand_real.move_to_joints(q_cmd, sim2real=False, t=0.2)  # move to init pose

In [ ]:
# rotation along z axis
f = 3
num = 10
t_total = num/f
q_cmd_list = generate_traj(w=-0.5, f=2, num=num)

hand_real.move_to_joints(q_cmd_list, t=t_total)
time.sleep(t_total)

q_cmd = q_sim2real(q0, legs=5)
hand_real.move_to_joints(q_cmd, sim2real=False, t=0.2)  # move to init pose



## For rotation movement

In [ ]:
## gait design for four legs
 # this is the init position


t0 = time.time()
r.f = 2
T = 1 / r.f
# v = np.array([0.0, -0.04,0]) 
# v = np.array([0.04,0 ,0]) 
# v = np.array([0, -0.02,0])
v = np.zeros(3)
w =  0.5 # rad/s
h = 0.02  # z axis lifting height for swing legs
r.vel_cmd[:3] = v

# xy_step = v[:2] * T/2 * 0.5  # based the MIT paper
xy_step = []
theta = w * T/2 * 0.5
R = np.array([[np.cos(theta), -np.sin(theta)], 
              [np.sin(theta), np.cos(theta)]])
xy_step = [xh_local_init[i][:2] @ R - xh_local_init[i][:2] for i in range(5)]
    
    
x_cmd = copy.deepcopy(xh_local_init)
x_cmd = [a[:3] for a in x_cmd]  # position command for legs
q_cmd_list = []
t_total = 40 * T

k = 0
q_desired = copy.deepcopy(r.q)
while 1:
    
    tau_Grf = np.zeros(20)
    t_now  = time.time() - t0
    t_period = t_now % (1/r.f)  # [0, T]
    
    if t_period < T/2: # FL and BR, front left and back right legs move during [0, T/2]
        swing_legs = [0, 2, 3]
        stance_legs = [1, 4]
        # swing_legs = [0, 1]
        # stance_legs = [3, 4]
    else:
        swing_legs = [1, 4]
        stance_legs = [0, 2, 3]
        # swing_legs = [3, 4]
        # stance_legs = [0, 1]
    
    # update vel direction in hand base frame
    # xy_step = rot.quat2mat(r.x[3:]).T @ np.array([xy_step[0], xy_step[1], 0])
    # xy_step = xy_step[:2]
    for i in swing_legs:
        t_tmp = t_period % (T/2)
        x_cmd[i][:2] = r.bezier(t_tmp, xh_local_init[i][:2] - xy_step[i], xh_local_init[i][:2] + xy_step[i])
        if t_tmp <= T/4:
            x_cmd[i][2] = r.bezier(t_tmp /(T/4), xh_local_init[i][2], xh_local_init[i][2] + h)  # z axis
        else:
            x_cmd[i][2] = r.bezier((t_tmp - T/4) /(T/4), xh_local_init[i][2] + h, xh_local_init[i][2])
    
    for i in stance_legs:
        t_tmp = t_period % (T/2)
        x_cmd[i][:2] = r.bezier(t_tmp /(T/2), xh_local_init[i][:2] + xy_step[i], xh_local_init[i][:2] - xy_step[i])
        x_cmd[i][2] = xh_local_init[i][2]
    
    q_desired = r.hand_kine.ik(x_cmd, q_init=q_desired)
    r.joint_computed_torque_control(q_desired)
    # k += 1
    # q_desired[8:12] = q0[8:12]
    q_cmd = q_sim2real(q_desired, legs=5)
    q_cmd_list.append(q_cmd)
    # hand_real.send_position(q_cmd)

    if t_now > t_total:
        break
    